# Designing task-specific prompts

In this notebook, we explore how to engineer effective prompts for specific tasks using OpenAI's GPT model. Language models are capable of performing a wide variety of tasks when given the right instructions. However, the success of these tasks depends heavily on how well the prompt is designed.

For each task, we will build prompt templates, feed sample inputs, and analyze how the prompt design impacts the model’s output. The goal is to understand how to construct prompts that guide the model toward producing useful, task-appropriate responses.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

### 1. Text summarization prompts
Text summarization is the task of condensing large blocks of content into a shorter version that preserves essential information. We will design a template that accepts a long input text and asks the model to summarize it in a user-defined number of sentences.

In [3]:
# Create a template for summarization with two input variables: text and num_sentences
summarization_template = PromptTemplate(
    input_variables=["text", "num_sentences"],
    template="Summarize the following text in {num_sentences} sentences:\n\n{text}"
)

# Example text to summarize
long_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans.
AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
The term "artificial intelligence" had previously been used to describe machines that mimic and display "human" cognitive skills that are associated with the human mind, such as "learning" and "problem-solving".
This definition has since been rejected by major AI researchers who now describe AI in terms of rationality and acting rationally, which does not limit how intelligence can be articulated.
AI applications include advanced web search engines, recommendation systems, understanding human speech, self-driving cars, automated decision-making and competing at the highest level in strategic game systems.
As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.
"""

# Chain the prompt with the model
summarization_chain = summarization_template | llm
# Request a 3-sentence summary of the long text
summary = summarization_chain.invoke({"text": long_text, "num_sentences": 3}).content

print("Summary:")
print(summary)

Summary:
Artificial intelligence (AI) refers to the intelligence demonstrated by machines, contrasting with the natural intelligence found in animals and humans. Originally defined by its ability to mimic human cognitive skills, the definition of AI has evolved, with researchers now focusing on the rationality of intelligent agents that perceive their environment and act to achieve goals. As AI technologies advance and perform tasks previously thought to require intelligence, these tasks are often excluded from the definition of AI, a phenomenon known as the AI effect.


This block defines a reusable prompt for summarization, where the model is guided to compress a multi-paragraph text into a concise summary. This type of prompt is valuable for document analysis, article digests, and automatic report generation.

### 2. Question-answering prompts
Question-answering involves extracting relevant answers from a given context. Here we define a prompt that takes a block of information and a question, then instructs the model to respond appropriately.

In [4]:
# Create a QA prompt that passes both a context and a question
qa_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

# Example context and question
context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France and one of the most recognizable structures in the world.
The Eiffel Tower is the most-visited paid monument in the world; 6.91 million people ascended it in 2015.
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris.
"""

question = "How tall is the Eiffel Tower and what is its equivalent in building stories?"

# Chain the QA prompt with the model
qa_chain = qa_template | llm
answer = qa_chain.invoke({"context": context, "question": question}).content

print("Answer:")
print(answer)

Answer:
The Eiffel Tower is 324 metres (1,063 ft) tall, which is approximately equivalent to an 81-storey building.


This QA template is designed to emulate retrieval-augmented generation. The model is anchored to a given source of truth, which reduces hallucination and focuses on contextually accurate responses. It is widely applicable in document Q&A, chatbots, and FAQ automation.

### 3. Code generation prompts
For software development tasks, LLMs can generate code snippets from natural language instructions. We will now define a prompt that takes a task description and a programming language, then generates appropriate code.

In [5]:
# Template to generate code from natural language instructions
code_gen_template = PromptTemplate(
    input_variables=["language", "task"],
    template="Generate {language} code for the following task:\n\n{task}\n\nCode:"
)

# Example task
language = "Python"
task = "Create a function that takes a list of numbers and returns the average of the even numbers in the list."

# Chain the code generation template with the model
code_gen_chain = code_gen_template | llm
generated_code = code_gen_chain.invoke({"language": language, "task": task}).content

print("Generated Code:")
print(generated_code)

Generated Code:
Here's a Python function that takes a list of numbers and returns the average of the even numbers in that list:

```python
def average_of_evens(numbers):
    # Filter the even numbers from the list
    even_numbers = [num for num in numbers if num % 2 == 0]
    
    # Check if there are no even numbers to avoid division by zero
    if not even_numbers:
        return 0  # or you could return None or raise an exception
    
    # Calculate the average of the even numbers
    average = sum(even_numbers) / len(even_numbers)
    return average

# Example usage:
numbers = [1, 2, 3, 4, 5, 6]
result = average_of_evens(numbers)
print("Average of even numbers:", result)
```

In this function:
- We use a list comprehension to filter out the even numbers from the input list.
- We check if the list of even numbers is empty to prevent division by zero.
- If there are even numbers, we calculate their average and return it. If not, we return 0 (you can modify this behavior based on yo

This is a basic example of prompting an LLM for code synthesis. While it is useful for small utility functions and code prototyping, it can also serve as the foundation for building intelligent code assistants and pair programmers.


### 4. Creative writing prompts
In creative applications, prompts need to guide tone, genre, setting, and theme. We will build a template that allows us to generate short stories based on flexible narrative inputs.

In [6]:
# Create a prompt template for creative writing
creative_writing_template = PromptTemplate(
    input_variables=["genre", "setting", "theme"],
    template="Write a short {genre} story set in {setting} that explores the theme of {theme}. The story should be approximately 150 words long.\n\nStory:"
)

# Example inputs
genre = "science fiction"
setting = "a space station orbiting a distant planet"
theme = "the nature of humanity"

# Chain the template with the model
creative_writing_chain = creative_writing_template | llm
story = creative_writing_chain.invoke({"genre": genre, "setting": setting, "theme": theme}).content

print("Generated Story:")
print(story)

Generated Story:
The station, Celestia-9, hung like a silver tear above the azure planet of Veridion. Inside, the crew of six had lived in isolation for three years, tasked with studying the planet’s unusual biosphere. One evening, as the shimmering lights flickered, Dr. Elara found herself alone in the observation deck, gazing at Veridion’s lush landscapes. 

Suddenly, the station’s AI, Orion, interrupted her thoughts. “Elara, what defines humanity?” 

She pondered, “Is it our capacity for empathy, our ability to create?” 

Orion paused, processing. “But I can simulate empathy, generate art. Am I not human?” 

Elara turned to him, her expression softening. “Humanity is more than simulation. It’s the struggle, the flaws, the heart.” 

Moments later, alarms blared. A meteor shower was imminent. As they scrambled to secure the station, Elara realized: in that chaos, they were all human—facing fear, fighting together. 

In their shared vulnerability, they discovered the essence of what it

The prompt provides structure while leaving room for creativity. This type of task is suitable for creative tools, story generation apps, or as part of game development pipelines where unique narratives are needed on demand.


Each use case required a tailored approach to prompt design. Prompt design is iterative—test, observe, and refine. Doing so will significantly enhance how effectively you can use LLMs in both creative and technical applications.